# Análise do Dataset: "Brazilian E-Commerce Public Dataset by Olist"
O Dataset da Olist é composto com informações de 100 mil pedidos, aproximadamente, realizados no período entre 2016 e 2018 em vários marketplaces no Brasil. 

Seus recursos permitem visualizar um pedido por várias dimensões: desde o status do pedido, preço, pagamento, desempenho do frete até a localização do cliente, entre outros. Para um melhor entendimento dos dados contidos no dataset, o conjunto é separado em diversos arquivos, onde cada um contém informações específicas.


Pode ser encontrado no link: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce


**Integrantes do grupo:**
- Guilherme Graboski
- Luiz Fonseca
- Matheus Aquino
- Pedro Petersen
- Rodolfo Ricardo
- Rosana Oliveira

________________________________________________________________________________

Importando as bibliotecas que usaremos:

In [ ]:
import pandas as pd
import numpy as np

 Para iniciar a análise, primeiramente vamos fazer a leitura de todos os arquivos e guardá-los em variáveis distintas:

In [ ]:
df_customers = pd.read_csv('/content/olist_customers_dataset.csv', sep=',')
df_order_items = pd.read_csv('/content/olist_order_items_dataset.csv', sep=',', decimal='.')
df_order_payments = pd.read_csv('olist_order_payments_dataset.csv', sep=',', decimal='.')
df_sellers = pd.read_csv('/content/olist_sellers_dataset.csv', sep=',')
df_products = pd.read_csv('/content/olist_products_dataset.csv', sep=',')
df_orders = pd.read_csv('/content/olist_orders_dataset.csv', sep=',')
df_order_reviews = pd.read_csv('/content/olist_order_reviews_dataset.csv', sep=',')

As linhas de código abaixo servem somente para imprimirem na tela as colunas dos dataset. Isso facilita para selecionarmos aquilo que gostaríamos de filtrar e com o que vamos trabalhar.

In [ ]:
print(list(df_customers.columns))
print(list(df_order_items.columns))
print(list(df_order_payments.columns))
print(list(df_sellers.columns))
print(list(df_products.columns))
print(list(df_orders.columns))
print(list(df_order_reviews.columns))

Pergunta principal: **Quais são as categorias de produtos mais propensas à insatisfação dos clientes?**

Primeiro, é necessário que nós façamos alguns 'merges' entre as diferentes planilhas, pegando somente as colunas que gostaríamos de trabalhar.

1º merge:

In [ ]:
# Separa somente as colunas que gostaríamos de dar merge
df_orders_col = df_orders[['order_id', 'order_status', 'order_delivered_customer_date','order_estimated_delivery_date']]

# Separa somente as colunas que gostaríamos de dar merge
df_order_items_col = df_order_items[['order_id', 'product_id', 'price', 'freight_value']] 
df_merged_0 = df_orders_col.merge(df_order_items_col, how='inner', on=['order_id'])

2º merge:

In [ ]:
df_products_col = df_products[['product_id', 'product_category_name']]
df_merged_1 = df_merged_0.merge(df_products_col, how='inner', on=['product_id'])

3º merge:

In [ ]:
df_order_reviews_col = df_order_reviews[['review_id', 'order_id', 'review_score']]
df_merged_2 = df_merged_1.merge(df_order_reviews_col, how='inner', on=['order_id'])

Vamos verificar como ficou o dataframe, após os três merges:

In [ ]:
df_merged_2

Explicação: foram feitos três .merge() seguidos para juntarmos as três planilhas desejadas para a análise que gostaríamos de fazer. Cada .merge() levou em consideração a 'key' que correlaciona as duas planilhas entre si.

Alguns detalhes são importantes para continuarmos a análise: entries, num. de linhas com dados e tipos das colunas. Vamos usar o .info()

In [ ]:
df_merged_2.info()

Após a verificação com o .info(), vemos que existem colunas com menos valores que o total de linhas e que algumas colunas precisam ser ajustadas para o tipo correto. 
As colunas com menos valores que a quantidade de linhas do dataset são: 'order_delivered_customer_date' e 'product_category_name'.

In [ ]:
list(df_merged_2['product_category_name'].unique())

Para os fins desta análise, os valores nulos (ou NaN) na coluna 'product_category_name' não serão considerados, uma vez que os produtos sem categoria definida representariam ruídos no dataset caso fossem acrescidos nas análises seguintes.

In [ ]:
df_merged_2 = df_merged_2[df_merged_2['product_category_name'].notna()]

**Explicação:** o comando acima retorna um dataset no qual somente as linhas com valores diferentes de None/Null/NaN, da coluna 'product_category_name', estão presentes.

In [ ]:
df_merged_2.info()

Já a coluna 'order_delivered_customer_date' representa a data que o consumidor recebeu o pedido em sua casa, ou seja, os pedidos com status de 'delivered'. 
Vamos verificar os diferentes status que os pedidos podem adquirir.

Podemos verificar que os possíveis status atribuídos a um pedido são os apresentados abaixo.

In [ ]:
list(df_merged_2['order_status'].unique())

Vamos verificar a porcentagem que cada status representa, do total de dados, dentro do dataset.

In [ ]:
df_merged_2.groupby('order_status')['order_status'].count() / df_merged_2.shape[0] * 100

Portanto, para os fins desta análise, somente o status de 'delivered' será considerado.

In [ ]:
df_merged_2 = df_merged_2[df_merged_2['order_status'] == 'delivered']

Verificando com o .info()

In [ ]:
df_merged_2.info()

Vemos que existem 8 pedidos que possuem o status de 'delivered', mas que não possuem data de entrega. Para que isso não cause ruídos na análise, vamos desconsiderar esses oito pedidos específicos.

In [ ]:
df_merged_2 = df_merged_2[df_merged_2['order_delivered_customer_date'].notna()]

Verificando novamente com o .info()

In [ ]:
df_merged_2.info()

Vemos, agora, que os pedidos restantes são aqueles que possuem o status de 'delivered' e possuem data de entreda do pedido ao cliente.

Depois de dropar todas as linhas com valores NaN e considerar somente os pedidos com status de 'delivered' e data de entrega, transformaremos o tipo das colunas 'order_delivered_customer_date' e 'order_estimated_delivery_date' em datetime.

In [ ]:
df_merged_2['order_delivered_customer_date'] = pd.to_datetime(df_merged_2['order_delivered_customer_date'],yearfirst=True)
df_merged_2['order_estimated_delivery_date'] = pd.to_datetime(df_merged_2['order_estimated_delivery_date'],yearfirst=True)

**Explicação:** alguns datasets possuem configurações de datas diferentes. Essa informação é importante, pois precisamos passar ao comando pd.to_datetime() a informação de yearfirst ou dayfirst, caso o ano venha primeiro ou o dia venha primeiro, respectivamente. No caso deste dataset, é yearfirst=True.

Verificando o dataset

In [ ]:
df_merged_2

Agora com o .info()

In [ ]:
df_merged_2.info()

Prosseguindo com a análise, lembramos que a pergunta que queremos responder é:

 **Quais são as categorias de produtos mais propensas à insatisfação dos clientes?**

Para isso, primeiramente vamos fazer uma análise da média do 'review_score' para cada categoria de produto.

Usando o .groupby() + .agg() para agrupar por categoria de produtos:
a média, a mediana e o desvio padrão são utilizadas para que se possa ter uma melhor abordagem dos resultados. 

In [ ]:
df_merged_2.groupby('product_category_name').agg({'review_score':['mean','std','median']}).sort_values(by=[('review_score','mean')])

De forma gráfica, apenas mostrando os valores apresentados acima:

In [ ]:
import seaborn as sns

df_plot = df_merged_2.groupby('product_category_name').mean().reset_index().sort_values('review_score')
sns.barplot(x = 'review_score', y = 'product_category_name', data = df_plot[:10], palette = 'rainbow')

Verificando a primeira categoria, que possui a menor média, podemos inferir que de acordo com o desvio padrão, os valores dos dados na amostra não são homogênio, ou seja, as reviews estão espalhadas dentro de 1 a 5 pontos, que são os valores possíveis para as reviews. 


Isso nos faz pensar no seguinte: **qual o motivo de alguns clientes darem a nota mais baixa e outros darem a nota mais alta para a mesma categoria de produto?** 

Vamos levar em consideração a categoria 'seguros_e_servicos'.

Agrupando pelo 'review_score', especificando a categoria de 'seguros_e_servicos', ou seja, queremos saber quantos deram 1 ponto, quantos deram 2, e assim por diante:

In [ ]:
df_merged_2[df_merged_2['product_category_name'] == 'seguros_e_servicos'].groupby('review_score')['review_score'].count()

Se calcularmos na mão, vemos que é exatamente isso: um pedido deu 1 ponto de 'review_score' e outro pedido deu 4 pontos de 'review_score'. Outra pergunta pode surgir na nossa cabeça: essa última categoria, 'fashion_roupa_infanto_juvenil' com média 5, quantos pedidos foram feitos nela e, consequentemente, qual a distribuição de votos (de 1 a 5) nesta categoria? Vamos verificar:

In [ ]:
df_merged_2[df_merged_2['product_category_name'] == 'fashion_roupa_infanto_juvenil'].groupby('review_score')['review_score'].count()

Considerando que categorias que possuem poucas compras (ou seja, amostra pequena de dados) podem nos fornecer resultados "mascarados", vamos então fazer média, desvio e  mediana para as cateogiras que mais tiveram pedidos.
Primeiro, vamos verificar quantos pedidos foram feitos em cada categoria.

In [ ]:
df_merged_2.groupby('product_category_name').agg({'order_id':['count']}).sort_values(by=[('order_id','count')], ascending=False) #.head(20)

De forma gráfica, para visualizarmos o quanto cada categoria impacta no total de pedidos:

In [ ]:
df_plot_2 = df_merged_2.groupby('product_category_name').count().reset_index().sort_values('order_id', ascending=False)
sns.barplot(x = 'order_id', y = 'product_category_name', data = df_plot_2[:10], palette = 'rainbow')

Vemos que as cinco maiores categorias possuem bastante dados (bastante pedidos), o que permite que façamos a análise para essas cinco categorias sem que os resultados sejam mascarados devido ao baixo número de pedidos. Logo, criando um novo dataframe apenas com as 20 maiores categorias:

In [ ]:
col_df_merged_2_20_maiores = list((df_merged_2.groupby('product_category_name').agg({'order_id':['count']}).sort_values(by=[('order_id','count')], ascending=False)).index[:20])
df_20_maiores_categorias = df_merged_2.loc[df_merged_2['product_category_name'].isin(col_df_merged_2_20_maiores)]
df_20_maiores_categorias

Agora, verificando a média, mediana e o desvio para as 20 categorias que mais tiveram pedidos:

In [ ]:
df_20_maiores_categorias.groupby('product_category_name').agg({'review_score':['mean','std','median']}).sort_values(by=[('review_score','mean')])

Como a categoria de 'moveis_escritorio' teve a menor média, vamos fazer a análise individual desta categoria.

In [ ]:
df_merged_2[df_merged_2['product_category_name'] == 'moveis_escritorio'].groupby('review_score')['review_score'].count()

Vemos que ela teve uma quantidade de votos 1 bastante significativa, em comparação com as demais. Portanto, ela poderia ser considerada a categoria mais propensa à insatisfação dos clientes (neste intervalo das 20 categorias com maior quantidade de pedidos).

Assim sendo, focando a análise na categoria de 'moveis_escritorio', vamos tentar descobrir qual a tendência para tantos votos 1 nesta categoria. Podemos considerar 2 pontos:
  1. Entrega atrasada
  2. Preço do frete alto

Inicialmente, para a Entrega Atrasada, temos que verificar quantos pedidos da categoria 'moveis_escritorio' foram entregues depois do prazo estimado de entrega.

Definindo um novo DataFrame apenas com a categoria de 'moveis_escritorio':

In [ ]:
df_merged_2_moveis_escritorios = df_merged_2[df_merged_2['product_category_name'] == 'moveis_escritorio']
df_merged_2_moveis_escritorios

Filtrando para as entregas atrasadas:

In [ ]:
df_entrega_atrasada = df_merged_2_moveis_escritorios[df_merged_2_moveis_escritorios['order_delivered_customer_date'] >= df_merged_2_moveis_escritorios['order_estimated_delivery_date']]
df_entrega_atrasada

Podemos ver acima os pedidos que tiveram entregas atrasadas. Mas quanto, desse total, é referente à voto 1, ou 2, ou 3...?

In [ ]:
df_entrega_atrasada.groupby('review_score')['review_score'].count()

Podemos perceber que os votos 1 e 2 somam 84, enquanto que 4 e 5 somam 45, que é pouco mais que a metade.
Porém, e em relação aos pedidos que foram entregues dentro do prazo? Vamos verificar.

Filtrando para os pedidos com a entrega antes do estimado:

In [ ]:
df_entrega_em_dia = df_merged_2_moveis_escritorios[df_merged_2_moveis_escritorios['order_delivered_customer_date'] <= df_merged_2_moveis_escritorios['order_estimated_delivery_date']]
df_entrega_em_dia.groupby('review_score')['review_score'].count()

Percebemos que a soma dos votos 1 e 2 é 339 enquanto que a soma dos votos 4 e 5 é 959, quase o triplo.

Ou seja, podemos dizer que, de certa forma, a entrega atrasa influencia sim no 'review_score' daquela categoria. Contudo, ainda podemos fazer outra análise.

Considerando, agora, o Preço do Frete Alto. Para isso, vamos estipular como "Alto" aquele frete cujo valor é maior ou igual a 50% do valor do pedido em si.

Filtrando os pedidos cujo frete é considerado Alto:

In [ ]:
df_frete_alto = df_merged_2_moveis_escritorios[df_merged_2_moveis_escritorios['freight_value'] > (df_merged_2_moveis_escritorios['price'] * 0.5)]
df_frete_alto

Verificando o total de voto no 'review_score':

In [ ]:
df_frete_alto.groupby('review_score')['review_score'].count()

Percebemos que desta vez a distribuição está bem homogênia, sendo que os votos 1 e 2 somam 44 e os votos 4 e 5 somam 62, valores bem próximos.

E para os fretes baixos (valor do frete menos que 50% do valor do pedido)?

In [ ]:
df_frete_baixo = df_merged_2_moveis_escritorios[df_merged_2_moveis_escritorios['freight_value'] < (df_merged_2_moveis_escritorios['price'] * 0.5)]
df_frete_baixo.groupby('review_score')['review_score'].count()

Agora, os votos 1 e 2 somam 381, enquanto que os votos 4 e 5 somam 942, mais uma vez quase o triplo. 

Porém, como os pedidos com frete considerados Altos não tiveram uma diferença muito significativa, não podemos inferir que o alto valor do frete tenha influência direta sobre o 'review_score' da categoria e, consequentemente, à insatisfação dos clientes quanto ao pedido.

Para finalizar, já que consideramos o 'review_score' e o tempo de entrega estimado do pedido, vamos apresentar um gráfico que mostra, de forma geral, qual a tendência de o consumidor atribuir uma nota baixa para o pedido de acordo com o tempo total que demora para o pedido ser entregue, a partir da data da compra.

Antes, alguns passos são necessários:

4º merge e transformação da coluna no tipo 'datetime':

In [ ]:
df_orders_col_2 = df_orders[['order_id','order_purchase_timestamp']]
df_merged_3 = df_merged_2.merge(df_orders_col_2, how='inner', on=['order_id']) 

df_merged_3['order_purchase_timestamp'] = pd.to_datetime(df_merged_3['order_purchase_timestamp'],yearfirst=True)

Cálculo do tempo de entrega do pedido, desde o momento da compra até o recebimento pelo consumidor:

In [ ]:
tempo_entrega = df_merged_3['order_delivered_customer_date'] - df_merged_3['order_purchase_timestamp'] 

Transformação do tempo de entrega em segundos, para facilitar o cálculo da quantidade total de dias para entrega:

In [ ]:
tempo_entrega_em_segundos = tempo_entrega.apply(lambda x: x.total_seconds()) 

Transformação de segundos em dias, para melhor interpretação, adicionando esta informação em uma nova coluna no dataframe:

In [ ]:
df_merged_3['tempo_entrega_dias'] = round(tempo_entrega_em_segundos / 86400, 2)

Visualizando o novo dataframe com a nova coluna:

In [ ]:
df_merged_3

Podemos mostrar, graficamente, o tempo de entrega (em dias) para todos os pedidos:

In [ ]:
sns.boxplot(df_merged_3['tempo_entrega_dias'], orient = 'h', showfliers = False, palette = 'flare')

Dessa forma, observando o gráfico, podemos perceber que o tempo de entrega do pedido, desde a data da compra até o recebimento do pedido pelo consumidor, fica entre 7 e 15 dias (aproximadamente).



> OBS: o erro que é dado ao executar o comando acima é facilmente corrigido explicitando o 'eixo x' do gráfico, ou seja, colocando o comando como:

> `sns.boxplot(x = df_merged_3['tempo_entrega_dias'], orient = 'h', showfliers = False, palette = 'flare').`

> Essa alteração será obrigatória em versões futuras, de acordo com o que é explicado no próprio log do erro.









Finalmente, respodendo à última pergunta: 

**Qual a tendência de o consumidor atribuir uma nota baixa para o pedido de acordo com o tempo total que demora para o pedido ser entregue, a partir da data da compra?**

In [ ]:
sns.boxplot(x = 'review_score', y = 'tempo_entrega_dias', data = df_merged_3, showfliers = False, palette = 'rainbow')

Portanto, utilizando o dataframe por completo e pelo gráfico abaixo, não podemos garantir que o tempo de entrega do produto tenha influência no 'review_score', mas é possível perceber que existe sim alguma correlação entre o review aplicado pelo consumidor e a demora no recebimento do pedido, à partir da data de compra.

Para mais informações sobre o que o seaborn pode fazer: https://seaborn.pydata.org/api.html

________________________________________________________________________________

Alguns notebooks que podem fornecer mais informações diversificadas e alguns insights sobre 'O que possível ser feito e ser analisado?' com esse dataset:


- [Geospatial Analysis of Brazilian E-Commerce](https://www.kaggle.com/code/andresionek/geospatial-analysis-of-brazilian-e-commerce?kernelSessionId=7894200)

- [E-Commerce Sentiment Analysis: EDA + Viz + NLP](https://www.kaggle.com/code/thiagopanini/e-commerce-sentiment-analysis-eda-viz-nlp?kernelSessionId=43455270)

- [Olist eCommerce-Analytics, Quasi Poisson+Poly Regs](https://www.kaggle.com/code/anshumoudgil/olist-ecommerce-analytics-quasi-poisson-poly-regs?kernelSessionId=28308821)


________________________________________________________________________________

**Demais perguntas que podem ser feitas e podem ser usadas para uma análise mais focada:**

## Pergunta 1: 
Qual estado tem mais clientes?

In [ ]:
df_customers.groupby(['customer_state'])['customer_state'].count().sort_values(ascending=False)

## Pergunta 2: 
Quais as 5 cidades que mais tiveram clientes?

In [ ]:
df_customers.groupby(['customer_city'])['customer_city'].count().sort_values(ascending=False).head()

## Pergunta 3: 
Quantas cidades tiveram uma única venda?

In [ ]:
(df_customers.groupby(['customer_city'])['customer_city'].count().sort_values(ascending=False) == 1).sum()


## Pergunta 4: 
Qual a média do valor do frete?

In [ ]:
df_order_items['freight_value'].mean()


## Pergunta 5: 
Qual a média do valor dos preços dos produtos?

In [ ]:
df_order_items['price'].mean()

## Pergunta 6: 
Qual o valor do produto mais caro?

In [ ]:
df_order_items['price'].max()

## Pergunta 7: 
Qual o valor do produto mais barato?

In [ ]:
df_order_items['price'].min()

## Pergunta 8: 
Qual foi a quantidade de vendas por forma de pagamento?

In [ ]:
df_order_payments.groupby(['payment_type'])['payment_type'].count().sort_values(ascending=False)

## Pergunta 9: 
Quais os cinco estados com mais vendedores?

In [ ]:
df_sellers.groupby('seller_state')['seller_state'].count().sort_values(ascending=False).head()


## Pergunta 10: 
Quais os cinco estados com menos vendedores?

In [ ]:
df_sellers.groupby('seller_state')['seller_state'].count().sort_values(ascending=False).tail()

________________________________________________________________________________

# Matheus

In [ ]:
df_orders = pd.read_csv('Trabalho/olist_orders_dataset.csv', sep=',', decimal='.')
df_orders = df_orders[['order_id','customer_id','order_status']]  #limpando colunas que não serão usadas

In [ ]:
df_orders.query("order_status == 'delivered'", inplace=True) #selecionando apenas as entregas com status delivered
df_orders

In [ ]:
df_order_items = pd.read_csv('Trabalho/olist_order_items_dataset.csv', sep=',', decimal='.')

In [ ]:
df_order_items = df_order_items[['order_id','order_item_id','product_id','price','freight_value']] #limpando colunas que não serão usadas

In [ ]:
df_order_payment = pd.read_csv('Trabalho/olist_order_payments_dataset.csv', sep=',', decimal='.')
df_order_payment = df_order_payment[['order_id','payment_type','payment_installments']] #limpando as colunas que não serão usadas

In [ ]:
df_customer = pd.read_csv('Trabalho/olist_customers_dataset.csv', sep=',', decimal='.')
df_customer = df_customer[['customer_id','customer_unique_id','customer_state']]  #limpando as colunas que não serão usadas

In [ ]:
df_merge_0 = df_orders.merge(df_order_items, how = 'inner', on =['order_id'])

In [ ]:
df_merge_1 = df_merge_0.merge(df_order_payment, how ='inner', on = ['order_id'])

In [ ]:
df_merge_2 = df_customer.merge(df_merge_1, how='left', on = ['customer_id'])

In [ ]:
df_merge_2.info()
df_analisada = df_merge_2[df_merge_2['order_id'].notna()]


In [ ]:
df_analisada['valor_total'] = df_analisada['price'] + df_analisada['freight_value'] #criando a coluna de valor total pago

In [ ]:
df_analisada['valor_prestacao'] = df_analisada['price']/df_analisada['payment_installments']  #criando coluna de parcela, nao leva em consideracao frete

In [ ]:
df_analisada

In [ ]:
df_analisada['customer_state'].isna()

In [ ]:
df_analisada.groupby('customer_state').agg({'valor_prestacao':['mean','std','median']})